In [16]:
import pandas as pd

df = pd.read_csv("valid_reports.csv", sep=",")
df["filepath"].tolist()

['/root/data_volumes/dataset/valid/valid_1/valid_1_a/valid_1_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_2/valid_2_a/valid_2_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_3/valid_3_a/valid_3_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_4/valid_4_a/valid_4_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_4/valid_4_b/valid_4_b_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_5/valid_5_a/valid_5_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_6/valid_6_a/valid_6_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_7/valid_7_a/valid_7_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_7/valid_7_b/valid_7_b_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_8/valid_8_a/valid_8_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_9/valid_9_a/valid_9_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_10/valid_10_a/valid_10_a_1.nii.gz',
 '/root/data_volumes/dataset/valid/valid_11/valid_11_a/valid_11_a_1.nii.gz',
 '/root/data_volumes/dataset/val

In [15]:
import json
import os
import pandas as pd

base_path = "/root/data_volumes/dataset/"

# Load JSON file
with open("valid_reports.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract rows
rows = [entry["row"] for entry in data["rows"]]

# Convert to DataFrame
df = pd.DataFrame(rows)


# Function to generate correct file path
def get_full_path(volume_name):
    parts = volume_name.split(
        "_"
    )  # Split "valid_1_a_1.nii.gz" → ['valid', '1', 'a', '1.nii.gz']
    if len(parts) < 4 or "2" in parts[3]:
        return None  # Fallback if format is unexpected

    base = parts[0]
    base_no = f"{base}_{parts[1]}"  # e.g., "valid_1"
    base_no_version = f"{base_no}_{parts[2]}"  # e.g., "valid_1_a"

    return os.path.join(base_path, base, base_no, base_no_version, volume_name)


# Add file path to VolumeName
df["VolumeName"] = df["VolumeName"].apply(get_full_path)

# Remove rows where VolumePath is None (i.e., those that had "_b")
df = df.dropna(subset=["VolumeName"])
df.rename(columns={"VolumeName": "filepath"}, inplace=True)
df.rename(columns={"Impressions_EN": "title"}, inplace=True)
df.drop(columns=["ClinicalInformation_EN", "Technique_EN", "Findings_EN"], inplace=True)

# Convert to CSV
df.to_csv("valid_reports.csv", index=False)